<img src="https://github.com/gantian127/nwm/blob/master/docs/source/_static/nwm_logo.png?raw=true" width='320' align='center'></a>

## Basic Info
This Jupyter Notebook was created by Tian Gan. You can test this Jupyter Notebook through [Binder](https://mybinder.org/v2/gh/gantian127/nwm/master?filepath=notebooks%2Fnwm.ipynb) or [HydroShare](http://www.hydroshare.org/resource/87fa0749a0b944228e3c613dc7d8899b). 

If you have any suggestion to improve the current functions of the nwm package, please create a github issue [here](https://github.com/gantian127/nwm/issues). 

Suggested citation: Gan, T. (2020). Jupyter Notebook for the nwm Python package, HydroShare, http://www.hydroshare.org/resource/87fa0749a0b944228e3c613dc7d8899b

## Quick Start Tutorial 

This notebook will help you get started using the nwm package to download the National Water Model (NWM) datasets. 

This tutorial includes the following sections:

1. [Brief Introduction](#section1)

   This section provides basic information about nwm package. 
   <br>
   
2. [Start with Examples](#section2)
   
   This section provides two examples to demonstrate how to use nwm to download datasets for visualization.
   <br>
   
3. [Write Your Own Code](#section3)

   This section provides guide to write your own code and explore the NWM datasets for hurricane events. 
   <br>
   

<a id='section1'></a>
## 1. Brief Introduction

nwm package provides a set of functions that allows downloading of the National Water Model ([NWM](https://water.noaa.gov/about/nwm)) datasets for data analysis and visualization. These functions were implemented using the API of the HydroShare National Water Model Web App. A HydroShare [account](https://www.hydroshare.org/sign-up/) is required to access [this app](https://hs-apps.hydroshare.org/apps/nwm-forecasts/).

nwm package also includes a Basic Model Interface ([BMI](https://bmi.readthedocs.io/en/latest/)), which converts the NWM dataset into a reusable, plug-and-play data component for [PyMT](https://pymt.readthedocs.io/en/latest/?badge=latest) modeling framework developed by Community Surface Dynamics Modeling System ([CSDMS](https://csdms.colorado.edu/wiki/Main_Page)) 

To install nwm package, you can use the following command:
```
$ pip install nwm
```

<a id='section2'></a>
## 2. Start with Examples

In nwm package, NwmHs class is designed for users to download datasets. BmiNwmHs class is designed to convert NWM dataset as a data component for the [PyMT](https://pymt.readthedocs.io/en/latest/?badge=latest) modeling framework. The following examples demonstrate how to download the same dataset using NwmHs and BmiNwmHs for data visualization. 

### Example 1: use NwmHs class to download data (Recommended method)

Import NwmHs class and download data with **get_data( )** method. This example downloads short range (18 hours) forecast  of streamflow at a river channel during a hurricane event. You can check the details of the [parameter settings](https://nwm.readthedocs.io/en/latest/#parameter-settings) for get_data( ) method to better understand the parameter values used in the example.

In [ ]:
import matplotlib.pyplot as plt
from nwm import NwmHs

# download streamflow data
nwm_data = NwmHs()
dataset = nwm_data.get_data(archive='harvey', config='short_range', geom='channel_rt',
                            variable='streamflow', comid=[5781915], init_time=0, 
                            start_date='2017-08-23')

The downloaded NWM dataset is stored as a self-described xarray object. With this data object, you can check the metadata and make time series plot.

In [ ]:
# show metadata
dataset.attrs

In [ ]:
# plot data
plt.figure(figsize=(9,5))
dataset.plot()
plt.xlabel('Year 2017')
plt.ylabel('{} ({})'.format(dataset.variable_name,dataset.variable_unit))
plt.title('Short range streamflow forecast for Channel 5781915 during Harvey Hurricane Event')

### Example 2: use BmiNwmHs class to download data (Demonstration of how to use BMI)

Import BmiNwmHs class and instantiate it. A configuration file (yaml file) is required to provide the parameter settings for data download. An example configure_file.yaml file is provided in the same folder with this Jupyter Notebook file. You can also skip this example and try section 3 to use NwmHs class for data download.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cftime

from nwm import BmiNwmHs

# initiate a data component
data_comp = BmiNwmHs()
data_comp.initialize('config_file.yaml')

Use variable related methods from BmiNwmHs class to check the variable information of the NWM dataset. This data component stores a flow forecast variable. 

In [ ]:
# get variable info
var_name = data_comp.get_output_var_names()[0]
var_unit = data_comp.get_var_units(var_name)
print(' variable_name: {}\n var_unit: {}\n'.format(var_name, var_unit))

Use time related methods of BmiNwmHs class to check the time information of the NWM dataset. The time values are stored in a format which follows [CF convention](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.pdf). 

In [ ]:
# get time info
start_time = data_comp.get_start_time()
end_time = data_comp.get_end_time()
time_step = data_comp.get_time_step()
time_unit = data_comp.get_time_units()
time_steps = int((end_time - start_time)/time_step) + 1
print(' start_time:{}\n end_time:{}\n time_step:{}\n time_unit:{}\n time_steps:{}\n'.format(start_time, end_time, time_step, time_unit, time_steps))

Loop through each time step to get the flow and time values. stream_array stores flow forecast values. cftime_array stores the numerical time values. time_array stores the corresponding  Python datetime objects. get_value( ) method returns the flow forecast value at each time step. update( ) method updates the current time step of the data component.

In [ ]:
# initiate numpy arrays to store data
stream_value = np.empty(1)
stream_array = np.empty(time_steps)
cftime_array = np.empty(time_steps)

for i in range(0, time_steps):
    data_comp.get_value(var_name, stream_value)
    stream_array[i] = stream_value
    cftime_array[i] = data_comp.get_current_time()
    data_comp.update()
    
time_array = cftime.num2date(cftime_array, time_unit, only_use_cftime_datetimes=False, only_use_python_datetimes=True)

In [ ]:
# plot data
plt.figure(figsize=(9,5))
plt.plot(time_array, stream_array)
plt.xlabel('Year 2017')
plt.ylabel('{} ({})'.format(var_name, var_unit))
plt.title('Short range streamflow forecast for Channel 5781915 during Harvey Hurricane Event')

<a id='section3'></a>
## 3. Write Your Own Code 

NwmHs class can be used to download datasets for several hurricane events, including [Harvey (2017)](https://en.wikipedia.org/wiki/Hurricane_Harvey), [Irma (2017)](https://en.wikipedia.org/wiki/Hurricane_Irma), and [Florence (2018)](https://en.wikipedia.org/wiki/Hurricane_Florence). Try with the following instructions and write your own code to explore the data for Hurricane Harvey. You can check the [parameter settings](https://nwm.readthedocs.io/en/latest/#parameter-settings) for get_data( ) method to download various climate forcing and forecast datasets.

### Hurricane Harvey 

Hurricane Harvey made landfall on Texas and Louisiana in August 2017. It caused catastrophic flooding and many deaths. In this use case, we will explore the datasets for Nederland, Texas, an area impacted by flooding during the hurricane event.

In [ ]:
import matplotlib.pyplot as plt
from nwm import NwmHs

nwm_data = NwmHs()

**Step1: Explore climate forcing data**

Download the rain rate data of a grid cell (comid=[833, 2596]) which locates in the Nederland area. The data is **short range** forecast for 2017-08-28 with model initiation time as 00:00.

In [ ]:
# download rain rate data
rain_rate = 

*Double-click __here__ for the solution.*

<!-- Your answer is below:

rain_rate = nwm_data.get_data(archive='harvey', config='short_range', geom='forcing',
                              variable='RAINRATE', comid=[833, 2596], init_time=0, 
                              start_date='2017-08-28')

-->

Plot the rain rate data. Heavy rainfall is usually more than 0.30 in/hr. What is the max rain rate for the grid area on that day?

In [ ]:
# plot rain rate data
rain_rate.plot()
plt.ylabel('{} ({})'.format(rain_rate.variable_name,rain_rate.variable_unit))
plt.title('Rain rate for grid cell [833, 2596]')

Download the air temperature data for the same grid cell (comid=[833, 2596]). Let's get this climate forcing data for **medium range** forecast starting from 2017-08-28 with model initiation time as 00:00.

In [ ]:
# download air temperature data
air_temp = 

*Double-click __here__ for the solution.*

<!-- Your answer is below:

air_temp = nwm_data.get_data(archive='harvey', config='medium_range', geom='forcing',
                             variable='T2D', comid=[833, 2596], init_time=0, 
                             start_date='2017-08-28')
-->

Plot the air temperature data. What is the max and min air temperature for the grid area during 2017-08-28 to 2017-09-27?

In [ ]:
# plot air temperature data
air_temp.plot()
plt.ylabel('{} ({})'.format(air_temp.variable_name,air_temp.variable_unit))
plt.title('Air temperature for grid cell [833, 2596]')

**Step2: Explore streamflow forecast results**

Download the streamflow data for a river channel (comid=[1112323]) which locates in the grid cell area. First try to get the **short range** forecast for 2017-08-28 with model initiation time as 00:00.  

In [ ]:
# download short range streamflow forecast 
short_range = 

*Double-click __here__ for the solution.*

<!-- Your answer is below:
    
short_range = nwm_data.get_data(archive='harvey', config='short_range', geom='channel_rt',
                                variable='streamflow', comid=[1112323], init_time=0, 
                                start_date='2017-08-28')

-->

Download the streamflow data for the same river channel (comid=[1112323]). Let's get the **medium range** and **long range** forecast datasets starting from 2017-08-28 with model initiation time as 00:00.  

In [ ]:
# download medium range streamflow forecast
medium_range = 

# download long range streamflow forecast
long_range = 

*Double-click __here__ for the solution.*

<!-- Your answer is below:
    
medium_range = nwm_data.get_data(archive='harvey', config='medium_range', geom='channel_rt',
                                 variable='streamflow', comid=[1112323], init_time=0, 
                                 start_date='2017-08-28')

long_range = nwm_data.get_data(archive='harvey', config='long_range', geom='channel_rt',
                               variable='streamflow', comid=[1112323], time_lag=0, 
                               start_date='2017-08-28')

-->

Download **analysis and assimilation** streamflow data from 2017-08-28 to 2017-09-10 for the same river channel (comid=[1112323]). The analysis and assimilation configuration produces a real-time analysis of the streamflow. The stream-gauge observations are assimilated from the USGS.

In [ ]:
# download analysis and assimilation streamflow data
analysis_assim = 

*Double-click __here__ for the solution.*

<!-- Your answer is below:
analysis_assim = nwm_data.get_data(archive='harvey', config='analysis_assim', 
                                   geom='channel_rt', variable='streamflow', comid=[1112323], 
                                   start_date='2017-08-28', end_date='2017-09-10')
-->

Let's plot all the streamflow datasets. From this plot, what do you find from the results produced by the four configurations?  

In [ ]:
# plot the streamflow forecast data
plt.figure(figsize=(14,7))
short_range.plot()
medium_range.plot()
long_range.plot()
analysis_assim.plot()

plt.ylabel('{} ({})'.format(short_range.variable_name,short_range.variable_unit))
plt.title('Streamflow forecast and analysis for Channel 1112323')
plt.legend(labels=['short range','medium range','long range','analysis assimilation'])

**Step3: Explore other forecast results**

Aside from streamflow data, NWM provides forecasts for other variables (e.g., soil moisture and groundwater runoff). We will download and visualize some of these datasets.

Download the near surface soil saturation data for the same grid cell (comid=[833, 2596]). The data is **short range** forecast for 2017-08-28 with model initiation time as 00:00.

In [ ]:
# download near surface soil saturation data
soil_sat = 

*Double-click __here__ for the solution.*

<!-- Your answer is below:

soil_sat = nwm_data.get_data(archive='harvey', config='short_range', geom='land',
                             variable='SOILSAT_TOP', comid=[833, 2596], init_time=0, 
                             start_date='2017-08-28')
-->

Plot the soil saturation data. It can be found that the near surface soil is highly saturated during heavy rainfall period (check with the rain rate plot above). 

In [ ]:
# plot near surface soil saturation data
soil_sat.plot()
plt.ylabel('soil saturation (m^3/m^3)')
plt.title('Near surface soil saturation for grid cell [833, 2596]')

Download the accumulated groundwater runoff data for the same grid cell (comid=[833, 2596]). The data is **long range** forecast starting from 2017-08-28 with model initiation time as 00:00.

In [ ]:
# download accumulated groundwater runoff data
groundwater_runoff = 

*Double-click __here__ for the solution.*

<!-- Your answer is below:

groundwater_runoff = nwm_data.get_data(archive='harvey', config='long_range', geom='land',
                                       variable='UGDRNOFF', comid=[833, 2596], time_lag=0, 
                                       start_date='2017-08-28')

-->

Plot the accumulated groundwater runoff data. How much groundwater runoff was generated during Aug 28-30th? 

In [ ]:
# plot accumulated groundwater runoff data
groundwater_runoff.plot()
plt.ylabel('{} ({})'.format(groundwater_runoff.variable_name,groundwater_runoff.variable_unit))
plt.title('Accumulated groundwater runoff for grid cell [833, 2596]')